## Feature Engineering

In [1]:
#imports

from __future__ import print_function
print('Print is ready to serve')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Print is ready to serve


In [9]:
df = pd.read_csv('./cleaned_transactions.csv')

In [10]:
df.shape

(33698, 8)

In [11]:
df.columns

Index([u'InvoiceNo', u'StockCode', u'Description', u'Quantity', u'InvoiceDate',
       u'UnitPrice', u'CustomerID', u'Country'],
      dtype='object')

In [12]:
df.dtypes

InvoiceNo        int64
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID       int64
Country         object
dtype: object

In [25]:
# we have UnitPrice and Quantity.
# so create features Sales
df['Sales'] = df['Quantity'] * df['UnitPrice']

In [26]:
df.shape

(33698, 9)

In [27]:
df.columns

Index([u'InvoiceNo', u'StockCode', u'Description', u'Quantity', u'InvoiceDate',
       u'UnitPrice', u'CustomerID', u'Country', u'Sales'],
      dtype='object')

## Customer Level transaction Features

In [28]:
## find out the total unique transactions by each customer ID

invoice_data = df.groupby('CustomerID').InvoiceNo.agg({'total_transactions': 'nunique'})

In [29]:
invoice_data[:5]

,total_transactions
CustomerID,
12347,7
12348,4
12349,1
12350,1
12352,8


In [30]:
df.columns

Index([u'InvoiceNo', u'StockCode', u'Description', u'Quantity', u'InvoiceDate',
       u'UnitPrice', u'CustomerID', u'Country', u'Sales'],
      dtype='object')

In [31]:
## aggregate product data by customer
product_data = df.groupby('CustomerID').StockCode.agg({
        'total_products' : 'count',
        'total_unique_products' : 'nunique'
    })

In [32]:
product_data[:5]

,total_products,total_unique_products
CustomerID,,
12347,182,103
12348,31,22
12349,73,73
12350,17,17
12352,85,59


In [33]:
df.columns

Index([u'InvoiceNo', u'StockCode', u'Description', u'Quantity', u'InvoiceDate',
       u'UnitPrice', u'CustomerID', u'Country', u'Sales'],
      dtype='object')

In [34]:
## aggregate sales data by customer

sales_data = df.groupby('CustomerID').Sales.agg( {
                                                    'total_sales': 'sum',
                                                    'avg_product_value' : 'mean'
                                                })

In [35]:
sales_data[:5]

,avg_product_value,total_sales
CustomerID,,
12347,23.681319,4310.00
12348,57.975484,1797.24
12349,24.076027,1757.55
12350,19.670588,334.40
12352,29.482824,2506.04


In [36]:
## for each customer, based on invoice find the total cart_value

cart_data = df.groupby(['CustomerID', 'InvoiceNo']).Sales.agg({ 'cart_value' : 'sum'})

In [37]:
# reset the index so that they can form the data frame
cart_data.reset_index(inplace=True)

In [38]:
cart_data.head(5)

,CustomerID,InvoiceNo,cart_value
0,12347,537626,711.79
1,12347,542237,475.39
2,12347,549222,636.25
3,12347,556201,382.52
4,12347,562032,584.91


In [39]:
## aggregate cart data at the customer level
agg_cart_data = cart_data.groupby(['CustomerID']).cart_value.agg({ 
            'avg_cart_value': 'mean',
            'min_cart_value': 'min',
            'max_cart_value': 'max'
    })

In [40]:
## combine all the data aggregations : invoice_data, product_data, sales_data, agg_cart_data
customer_df = invoice_data.join([product_data, sales_data, agg_cart_data])

In [41]:
customer_df.head(5)

,total_transactions,total_products,total_unique_products,avg_product_value,total_sales,min_cart_value,max_cart_value,avg_cart_value
CustomerID,,,,,,,,
12347,7,182,103,23.681319,4310.00,224.82,1294.32,615.714286
12348,4,31,22,57.975484,1797.24,227.44,892.80,449.310000
12349,1,73,73,24.076027,1757.55,1757.55,1757.55,1757.550000
12350,1,17,17,19.670588,334.40,334.40,334.40,334.400000
12352,8,85,59,29.482824,2506.04,120.33,840.30,313.255000


In [42]:
# customerID will be the index.
#
# Save analytical base table
customer_df.to_csv('analytical_base_table.csv')